# Notebook for meta-learning human motion from few random samples from a time series

# Meta-learning for regression tasks
1) Sinewaves

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import torch.nn as nn
import numpy as np
import glob
import learn2learn as l2l
from collections import OrderedDict
from human_body_prior.tools.omni_tools import copy2cpu as c2c
import warnings
from torch import optim
warnings.filterwarnings("ignore")

import trimesh
from human_body_prior.tools.omni_tools import colors
from human_body_prior.mesh import MeshViewer
from human_body_prior.mesh.sphere import points_to_spheres
from notebook_tools import show_image

In [2]:
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def loss_meshes(target, predicted_poses, loss_fn, shots, bm):
    loss = torch.zeros(1)
    betas = target['betas']
    dmpls = target['dmpls']
    poses = target['poses']
    for i in range(shots):
        #print("i: ", i)
      #  print(predicted_poses[i].unsqueeze(0).shape)
       # print(betas.shape)
        #print(dmpls[i].shape)
        target = bm(pose_body=poses[i,3:66].unsqueeze(0), betas=betas, dmpls=dmpls[i].unsqueeze(0))
        mesh = bm(pose_body = predicted_poses[i].unsqueeze(0), betas=betas, dmpls=dmpls[i].unsqueeze(0))
        loss = loss + torch.abs(target.v - mesh.v).mean()
    loss /=shots
    return loss

# Load data, Body model used SMPLH (body, hands and dmpls)

In [4]:
from human_body_prior.body_model.body_model import BodyModel

bm_path = '/home/michalislazarou/PhD/amass/smplh/male/model.npz'
dmpl_path = '/home/michalislazarou/PhD/amass/dmpl/male/model.npz'
npz_bdata_path ='/home/michalislazarou/PhD/amass/datasets/EKUT/234/'
num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_path=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, path_dmpl=dmpl_path).to(comp_device)
print("Body params: ", bm.pose_body.shape, "hand params: ", bm.pose_hand.shape, "dmpl: ", bm.dmpls.shape)
faces = c2c(bm.f)

Body params:  torch.Size([1, 63]) hand params:  torch.Size([1, 90]) dmpl:  torch.Size([1, 8])


# Create a Class to smaple Human motion examples from a dataset, in our case for this notebook we use the dataset EKUT(KIT Whole-Body Human Motion Database)

In [5]:
class HumanMotionDataset():
    def __init__(self, path_to_data, shots):
        self.img_path = path_to_data
        train_meshes, test_meshes = self.get_labels()
        self.train_meshes = train_meshes
        self.test_meshes = test_meshes
        self.shots = shots
        self.mesh = None
    
    def sample_task(self, train = 'train'):
        """ Sampling a task means sampling an mesh. """ 
        # choose image
        if train=='train':
            index = np.random.randint(len(self.train_meshes))
            self.mesh = self.train_meshes[index]
        else:
            index = np.random.randint(len(self.test_meshes))
            self.mesh = self.test_meshes[index]
        #return mesh
    
    def get_samples(self):
        x = np.sort(np.random.choice(self.mesh['poses'].shape[0], self.shots, replace=False))
        y = {}
        y['poses'] = self.mesh['poses'][x,:]
        y['trans'] = self.mesh['trans'][x,:]
        y['dmpls'] = self.mesh['dmpls'][x,:]
        y['betas'] = self.mesh['betas']
        return x, y
    def get_labels(self):
        data_list = []
        for data_file in glob.glob(os.path.join(self.img_path, '*.npz')):
            bdata = dict(np.load(data_file))
            bdata['poses'] = bdata['poses'][:200,:]
            bdata['trans'] = bdata['trans'][:200,:]
            bdata['dmpls'] = bdata['dmpls'][:200,:]
            data_list.append(bdata)
        return data_list[:70], data_list[70:] 

# In a similar way to the regression problem in sinewaves and image completion in: 
MAML:https://arxiv.org/pdf/1703.03400.pdf <br>
CAVIA:https://arxiv.org/pdf/1810.03642.pdf<br>
CNP: https://arxiv.org/pdf/1807.01613.pdf<br>
a regressor network is used as our meta-learning with the aim to regress the human motion from only a few samples of the complete motion. This can also be viewed as interpolation of motion.

In [6]:
class RegressorNet(nn.Module):
    def __init__(self):
        super(RegressorNet,self).__init__()
        self.net=nn.Sequential(OrderedDict([
            ('l1',nn.Linear(1,128)),
            ('relu1',nn.ReLU()),
            ('l2',nn.Linear(128,128)),
            ('relu2',nn.ReLU()),
            ('l3',nn.Linear(128,128)),
            ('relu3',nn.ReLU()),
            ('l4',nn.Linear(128,128)),
            ('relu4',nn.ReLU()),
            ('l5',nn.Linear(128,63)),
            #('relu5',nn.ReLU()),
            #('l6',nn.Linear(40,1))
        ]))
    
    def forward(self,x):
        return self.net(x)

# Load human motion dataset for single mesh example 

In [7]:
# import meta-dataset
data_list = []
for data_file in glob.glob(os.path.join(npz_bdata_path, '*.npz')):
    bdata = dict(np.load(data_file))
    bdata['poses'] = bdata['poses'][:200,:]
    bdata['trans'] = bdata['trans'][:200,:]
    bdata['dmpls'] = bdata['dmpls'][:200,:]
    data_list.append(bdata)
train_list, test_list = data_list[:70], data_list[70:] 
print(bdata['betas'].shape)
print(len(data_list), len(train_list), len(test_list))

(16,)
79 70 9


# Meta-learning hyperparameters initialized 

In [8]:
alpha=0.01
beta=0.001
shots=50
adaptation_steps=50
meta_batch_size = 10
num_iterations = 20000
regressor = RegressorNet()

# Single mesh regression:
Target mesh shown<br>
Note that only the pose parameters of the body are regressed, no pose parameters for hands or shape parameters

In [9]:
bdata = data_list[25]

fId = 30 # frame id of the mocap sequence

root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
dmpls = torch.Tensor(bdata['dmpls'][fId:fId+1]).to(comp_device) # controls soft tissue dynamics

body = bm(pose_body=pose_body, betas=betas, dmpls=dmpls)
target = trimesh.Trimesh(vertices=c2c(body.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
#target.show()
print(pose_body.shape)
print(betas.shape)
print(dmpls.shape)

torch.Size([1, 63])
torch.Size([1, 10])
torch.Size([1, 8])


# Initial mesh

In [10]:
#meta learnable parameters
body_pose = torch.tensor(np.zeros((1, 63)).astype('float32'), requires_grad = True, device = comp_device)
#hand_pose = torch.tensor(np.zeros((1, 90)).astype('float32'), requires_grad = True, device = device)
#dmpls = torch.tensor(np.zeros((1, 8)).astype('float32'), requires_grad = True, device = device)
#maml = l2l.algorithms.MAML(body_pose, lr=alpha, first_order=False)
loss=nn.MSELoss()
opt = optim.Adam(regressor.parameters(), alpha)
#maml = l2l.algorithms.MAML([body_pose], lr=alpha, first_order=False)
#opt = optim.Adam(maml.parameters(), beta)
mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
#body_mesh.show()
x=torch.tensor(fId, dtype=torch.float).unsqueeze(0)
#print(x.shape)
body_pose = regressor(x).unsqueeze(0)
mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
body_mesh.show()

# Learn the regressor parameters to predict the pose coefficients for the target mesh, iter=5000

In [11]:
for i in range(5000):
    
    body_pose = regressor(x).unsqueeze(0)
    
    #print(body_pose.shape, pose_body.shape)
    
    mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
    
    opt.zero_grad()
    
    loss = torch.abs(body.v - mesh.v).mean()
    
    loss.backward(retain_graph=True)
    
    opt.step()
    if i%200==0:
        print("iteration: ", i, "loss: ", loss.item())
   # mesh_view = smpl_model(beta.detach(), theta.detach())
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
print(body_mesh.center_mass)

iteration:  0 loss:  0.2668391466140747
iteration:  200 loss:  0.007803488057106733
iteration:  400 loss:  0.004052243195474148
iteration:  600 loss:  0.00248341984115541
iteration:  800 loss:  0.0018775655189529061
iteration:  1000 loss:  0.0016478390898555517
iteration:  1200 loss:  0.0008407389977946877
iteration:  1400 loss:  0.0015201870119199157
iteration:  1600 loss:  0.001487393630668521
iteration:  1800 loss:  0.0022950416896492243
iteration:  2000 loss:  0.0016861604526638985
iteration:  2200 loss:  0.0010083134984597564
iteration:  2400 loss:  0.0014089198084548116
iteration:  2600 loss:  0.001752345822751522
iteration:  2800 loss:  0.0016453891294077039
iteration:  3000 loss:  0.0017835001926869154
iteration:  3200 loss:  0.001953649567440152
iteration:  3400 loss:  0.001644213916733861
iteration:  3600 loss:  0.0014255242422223091
iteration:  3800 loss:  0.0014428318245336413
iteration:  4000 loss:  0.0009940855670720339
iteration:  4200 loss:  0.0006640315405093133
iterat

# Regressor network mesh prediction

In [12]:
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
body_mesh.show()

# Target mesh shown

In [13]:
target.show()

# Regress 50 meshes as a time series signal with the same regressor network.
This can be thought of as learning a parametrized function H from time, t, to mesh, M.<br>

$$H_{\phi}(t_i) \rightarrow M_{t_i}$$

In [14]:
human_motion = HumanMotionDataset(npz_bdata_path, shots)
human_motion.sample_task()
x, y = human_motion.get_samples()
x=torch.tensor(x, dtype=torch.float).unsqueeze(1)
#print(y['betas'].shape)
#convert to pytorch tensors
y['betas'] = torch.Tensor(y['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
y['trans'] = torch.Tensor(y['trans']).to(comp_device)
y['poses'] = torch.Tensor(y['poses']).to(comp_device)
y['dmpls'] = torch.Tensor(y['dmpls']).to(comp_device)
# use in learning loop
#target_pose = y['poses'][:,3:66]
#betas = y['betas']
#dmpls = y['dmpls']
print(y['betas'].shape)
print(y['poses'].shape)
print(y['dmpls'].shape)

torch.Size([1, 10])
torch.Size([50, 156])
torch.Size([50, 8])


In [ ]:
for i in range(50000):
    
    print(i)
    
    body_pose = regressor(x)
    
    #print(body_pose.shape)
    
    loss = loss_meshes(y, body_pose, nn.MSELoss(), shots, bm)
    
    #mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
    
    opt.zero_grad()
    
    #loss = torch.abs(body.v - mesh.v).mean()
    
    loss.backward(retain_graph=True)
    
    opt.step()
    if i%200==0:
        print("iteration: ", i, "loss: ", loss.item())

0
iteration:  0 loss:  0.07279682159423828
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
iteration:  200 loss:  0.02038741298019886
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
25